<a href="https://colab.research.google.com/github/wonkwonlee/a2w-kma-big-data-contest/blob/main/XGBoost_undersample_submission_1v30.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks')

In [ ]:
import tensorflow as tf
from tensorflow import keras

import os
import tempfile

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

import sklearn
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.linear_model import LogisticRegression


from xgboost import plot_importance
from xgboost import XGBClassifier

In [ ]:
def metrics(y_test, pred):
    accuracy = accuracy_score(y_test, pred)
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test, pred)
    f1 = f1_score(y_test, pred)
    roc_score = roc_auc_score(y_test, pred)
    print('정확도 : {0:.2f}, 정밀도 : {1:.2f}, 재현율 : {2:.2f}'.format(accuracy, precision, recall))
    print('f1-score : {0:.2f}, auc : {1:.2f}'.format(f1, roc_score))

# 데이터 처리 및 탐색

In [ ]:
raw_df = pd.read_csv("../A2W/데이터/data_submission/merge_final_real.csv", encoding='utf-8-sig')

In [ ]:
raw_df['1day_yn'] = 0.0
raw_df['2day_yn'] = 0.0

In [ ]:
# 열을 밀어서 출력, 맨 첫날, 마지막 날의 경우 산사태가 발생하지 않음.
raw_df['1day_yn'][1:-1] = raw_df['landslide'][0:-2]
raw_df['2day_yn'][2:-1] = raw_df['landslide'][0:-3]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
select_df = raw_df[['maxTa','minTa','avgTa','EIGHT_AGL','mi10MaxRn', 'hr1MaxRn', 'sumRnDur', 'sumRn', 'maxInsWs', 'maxWs', 'avgWs','minRhm','avgRhm','n99Rn', 'PRRCK_LARG', 'PRRCK_MDDL','LOCTN_ALTT','LOCTN_GRDN','CLZN_CD','TPGRP_TPCD','PRDN_FOM_C','SLDPT_TPCD','SLANT_TYP','SCSTX_CD','SLTP_CD','STORUNST','FROR_CD','FRTP_CD','KOFTR_GROU','DMCLS_CD','AGCLS_CD','HEIGHT','1day_yn']]
cleaned_df_1 = select_df.fillna(0)

## 1일 후 예측

In [ ]:
y = cleaned_df_1['1day_yn']
X = cleaned_df_1.drop('1day_yn', axis =1)

In [ ]:
from imblearn.under_sampling import RandomUnderSampler

# 1: 50 비율
X_samp, y_samp = RandomUnderSampler(random_state=1024, ratio={1: 359, 0: 10770}).fit_sample(X, y)

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_samp, y_samp, test_size=0.25, shuffle=True, stratify=y_samp, random_state=1024)

### 하이퍼 파라미터 튜닝 후 예측

In [ ]:
xgb = XGBClassifier()

xgb_param_grid = {
    'n_estimators' : [100, 200, 400, 500, 600],
    'learning_rate' : [0.01, 0.05, 0.1, 0.15, 0.2],
    'max_depth' : [4, 6, 8, 10, 12],
}
# sklearn.metrics.SCORERS.keys()
xgb_grid = GridSearchCV(xgb, param_grid = xgb_param_grid, scoring="roc_auc", n_jobs= 1, verbose = 1)
xgb_grid.fit(X_train, y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 5 folds for each of 125 candidates, totalling 625 fits


[Parallel(n_jobs=1)]: Done 625 out of 625 | elapsed: 34.9min finished


GridSearchCV(cv=None, error_score=nan,
             estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                     colsample_bylevel=1, colsample_bynode=1,
                                     colsample_bytree=1, gamma=0,
                                     learning_rate=0.1, max_delta_step=0,
                                     max_depth=3, min_child_weight=1,
                                     missing=None, n_estimators=100, n_jobs=1,
                                     nthread=None, objective='binary:logistic',
                                     random_state=0, reg_alpha=0, reg_lambda=1,
                                     scale_pos_weight=1, seed=None, silent=None,
                                     subsample=1, verbosity=1),
             iid='deprecated', n_jobs=1,
             param_grid={'learning_rate': [0.01, 0.05, 0.1, 0.15, 0.2],
                         'max_depth': [4, 6, 8, 10, 12],
                         'n_estimators': [100, 2

In [ ]:
print("최고 평균 auc : {0:.4f}".format(xgb_grid.best_score_))
print("최고의 파라미터 : ", xgb_grid.best_params_)
print("최고 점수를 낸 파라미터를 가진 모형 : ", xgb_grid.best_estimator_)

최고 평균 auc : 0.9944
최고의 파라미터 :  {'learning_rate': 0.1, 'max_depth': 6, 'n_estimators': 200}
최고 점수를 낸 파라미터를 가진 모형 :  XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=None, n_estimators=200, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)


In [ ]:
result = pd.DataFrame(xgb_grid.cv_results_)
result.sort_values(by=['rank_test_score'], inplace=True)

In [ ]:
result[['params', 'mean_test_score', 'rank_test_score']].head(10)

,params,mean_test_score,rank_test_score
56,"{'learning_rate': 0.1, 'max_depth': 6, 'n_esti...",0.994377,1
32,"{'learning_rate': 0.05, 'max_depth': 6, 'n_est...",0.994320,2
33,"{'learning_rate': 0.05, 'max_depth': 6, 'n_est...",0.994285,3
34,"{'learning_rate': 0.05, 'max_depth': 6, 'n_est...",0.994179,4
52,"{'learning_rate': 0.1, 'max_depth': 4, 'n_esti...",0.994092,5
76,"{'learning_rate': 0.15, 'max_depth': 4, 'n_est...",0.994071,6
53,"{'learning_rate': 0.1, 'max_depth': 4, 'n_esti...",0.994053,7
55,"{'learning_rate': 0.1, 'max_depth': 6, 'n_esti...",0.994031,8
54,"{'learning_rate': 0.1, 'max_depth': 4, 'n_esti...",0.994013,9
105,"{'learning_rate': 0.2, 'max_depth': 6, 'n_esti...",0.993959,10


In [ ]:
import joblib

xgb1 = xgb_grid.best_estimator_
xgb1.fit(X_train, y_train)
joblib.dump(xgb1, './xgb_1v30_model_1day_yn.pkl')

['./xgb_1v30_model_1day_yn.pkl']

In [ ]:
xgb1_pred = xgb1.predict(X_test)
metrics(y_test, xgb1_pred)

정확도 : 0.99, 정밀도 : 0.88, 재현율 : 0.82
f1-score : 0.85, auc : 0.91


## 2일 후 예측

In [ ]:
#분석을 위해 음수 값을 가지는 칼럼들을 제외하고 결측치를 0으로 대체한다.
select_df = raw_df[['maxTa','minTa','avgTa','EIGHT_AGL','mi10MaxRn', 'hr1MaxRn', 'sumRnDur', 'sumRn', 'maxInsWs', 'maxWs', 'avgWs','minRhm','avgRhm','n99Rn', 'PRRCK_LARG', 'PRRCK_MDDL','LOCTN_ALTT','LOCTN_GRDN','CLZN_CD','TPGRP_TPCD','PRDN_FOM_C','SLDPT_TPCD','SLANT_TYP','SCSTX_CD','SLTP_CD','STORUNST','FROR_CD','FRTP_CD','KOFTR_GROU','DMCLS_CD','AGCLS_CD','HEIGHT','2day_yn']]
cleaned_df_2 = select_df.fillna(0)

In [ ]:
y = cleaned_df_2['2day_yn']
X = cleaned_df_2.drop('2day_yn', axis =1)

In [ ]:
# 1: 50 비율
X_samp, y_samp = RandomUnderSampler(random_state=1024, ratio={1: 359, 0: 10770}).fit_sample(X, y)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_samp, y_samp, test_size=0.25, shuffle=True, stratify=y_samp, random_state=1024)

print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

X_train shape: (8346, 32)
X_test shape: (2783, 32)
y_train shape: (8346,)
y_test shape: (2783,)


In [ ]:
xgb2 = XGBClassifier(n_estimators=200, learning_rate = 0.05, max_depth = 10)
evals = [(X_test, y_test)]
xgb2.fit(X_train, y_train, eval_metric = "logloss", eval_set = evals, verbose = 1)

[0]	validation_0-logloss:0.646881
[1]	validation_0-logloss:0.605096
[2]	validation_0-logloss:0.56703
[3]	validation_0-logloss:0.532345
[4]	validation_0-logloss:0.500501
[5]	validation_0-logloss:0.471344
[6]	validation_0-logloss:0.444247
[7]	validation_0-logloss:0.419183
[8]	validation_0-logloss:0.396157
[9]	validation_0-logloss:0.374792
[10]	validation_0-logloss:0.354999
[11]	validation_0-logloss:0.336584
[12]	validation_0-logloss:0.319255
[13]	validation_0-logloss:0.303141
[14]	validation_0-logloss:0.288137
[15]	validation_0-logloss:0.274005
[16]	validation_0-logloss:0.26079
[17]	validation_0-logloss:0.248447
[18]	validation_0-logloss:0.236788
[19]	validation_0-logloss:0.225809
[20]	validation_0-logloss:0.215658
[21]	validation_0-logloss:0.205981
[22]	validation_0-logloss:0.196861
[23]	validation_0-logloss:0.188364
[24]	validation_0-logloss:0.180296
[25]	validation_0-logloss:0.172767
[26]	validation_0-logloss:0.165606
[27]	validation_0-logloss:0.158857
[28]	validation_0-logloss:0.1524

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.05, max_delta_step=0, max_depth=10,
              min_child_weight=1, missing=None, n_estimators=200, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [ ]:
xgb = XGBClassifier()

xgb_param_grid = {
    'n_estimators' : [100, 200, 400, 500, 600],
    'learning_rate' : [0.01, 0.05, 0.1, 0.15, 0.2],
    'max_depth' : [4, 6, 8, 10, 12],
}
# sklearn.metrics.SCORERS.keys()
xgb_grid = GridSearchCV(xgb, param_grid = xgb_param_grid, scoring="roc_auc", n_jobs= 1, verbose = 1)
xgb_grid.fit(X_train, y_train)

Fitting 5 folds for each of 125 candidates, totalling 625 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 625 out of 625 | elapsed: 37.0min finished


GridSearchCV(cv=None, error_score=nan,
             estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                     colsample_bylevel=1, colsample_bynode=1,
                                     colsample_bytree=1, gamma=0,
                                     learning_rate=0.1, max_delta_step=0,
                                     max_depth=3, min_child_weight=1,
                                     missing=None, n_estimators=100, n_jobs=1,
                                     nthread=None, objective='binary:logistic',
                                     random_state=0, reg_alpha=0, reg_lambda=1,
                                     scale_pos_weight=1, seed=None, silent=None,
                                     subsample=1, verbosity=1),
             iid='deprecated', n_jobs=1,
             param_grid={'learning_rate': [0.01, 0.05, 0.1, 0.15, 0.2],
                         'max_depth': [4, 6, 8, 10, 12],
                         'n_estimators': [100, 2

In [ ]:
print("최고 평균 auc : {0:.4f}".format(xgb_grid.best_score_))
print("최고의 파라미터 : ", xgb_grid.best_params_)
print("최고 점수를 낸 파라미터를 가진 모형 : ", xgb_grid.best_estimator_)

최고 평균 auc : 0.9915
최고의 파라미터 :  {'learning_rate': 0.2, 'max_depth': 4, 'n_estimators': 100}
최고 점수를 낸 파라미터를 가진 모형 :  XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.2, max_delta_step=0, max_depth=4,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)


In [ ]:
result = pd.DataFrame(xgb_grid.cv_results_)
result.sort_values(by=['rank_test_score'], inplace=True)

In [ ]:
result[['params', 'mean_test_score', 'rank_test_score']].head(10)

,params,mean_test_score,rank_test_score
100,"{'learning_rate': 0.2, 'max_depth': 4, 'n_esti...",0.991524,1
85,"{'learning_rate': 0.15, 'max_depth': 8, 'n_est...",0.991129,2
36,"{'learning_rate': 0.05, 'max_depth': 8, 'n_est...",0.991116,3
32,"{'learning_rate': 0.05, 'max_depth': 6, 'n_est...",0.990880,4
37,"{'learning_rate': 0.05, 'max_depth': 8, 'n_est...",0.990717,5
86,"{'learning_rate': 0.15, 'max_depth': 8, 'n_est...",0.990647,6
60,"{'learning_rate': 0.1, 'max_depth': 8, 'n_esti...",0.990639,7
42,"{'learning_rate': 0.05, 'max_depth': 10, 'n_es...",0.990583,8
38,"{'learning_rate': 0.05, 'max_depth': 8, 'n_est...",0.990566,9
61,"{'learning_rate': 0.1, 'max_depth': 8, 'n_esti...",0.990538,10


In [ ]:
import joblib

xgb2 = xgb_grid.best_estimator_
xgb2.fit(X_train, y_train)
joblib.dump(xgb2, './xgb_1v30_model_2day_yn.pkl')

['./xgb_1v30_model_2day_yn.pkl']

In [ ]:
xgb2_pred = xgb2.predict(X_test)
metrics(y_test, xgb2_pred)

정확도 : 0.99, 정밀도 : 0.96, 재현율 : 0.78
f1-score : 0.86, auc : 0.89


# Submission 실행

In [ ]:
# Merged_data_file
Merged_2020 = pd.read_csv('../A2W/데이터/data_submission/merge_s_h_a20.csv')
# Test 검증 데이터셋
test_input = pd.read_csv('../A2W/데이터/data_submission/input.csv', encoding = 'cp949')

In [ ]:
# 훈련된 모델로 결과 도출 : day1 함수
def day1_data_test(sgg, umd, date):    
    # sgg 데이터와 umd 데이터, date를 통한 추출
    text = sgg + ' ' + umd
    # specific 자체가 학습 돌린 결과 참고해서 input으로 만들어야 됨
    test_day1 = Merged_2020[Merged_2020['umd2'].str.contains(text.rstrip()) & Merged_2020['tm'].str.contains(date)]
    test_specific_day1 = test_day1[['maxTa','minTa','avgTa','EIGHT_AGL','mi10MaxRn', 'hr1MaxRn', 'sumRnDur', 'sumRn', 'maxInsWs', 
                                    'maxWs', 'avgWs','minRhm','avgRhm','n99Rn', 'PRRCK_LARG', 'PRRCK_MDDL','LOCTN_ALTT','LOCTN_GRDN',
                                    'CLZN_CD','TPGRP_TPCD','PRDN_FOM_C','SLDPT_TPCD','SLANT_TYP','SCSTX_CD','SLTP_CD','STORUNST','FROR_CD',
                                    'FRTP_CD','KOFTR_GROU','DMCLS_CD','AGCLS_CD','HEIGHT']]
    fillna_test_specific_day1 = test_specific_day1.fillna(0)
    model_pred = xgb1.predict(fillna_test_specific_day1.to_numpy())
    return model_pred

# 훈련된 모델로 결과 도출 : day2 함수
def day2_data_test(sgg, umd, date):    
    # sgg 데이터와 umd 데이터, date를 통한 추출
    text = sgg + ' ' + umd
    # specific 자체가 학습 돌린 결과 참고해서 input으로 만들어야 됨
    test_day2 = Merged_2020[Merged_2020['umd2'].str.contains(text.rstrip()) & Merged_2020['tm'].str.contains(date)]
    test_specific_day2 = test_day2[['maxTa','minTa','avgTa','EIGHT_AGL','mi10MaxRn', 'hr1MaxRn', 'sumRnDur', 'sumRn', 'maxInsWs',
                                    'maxWs', 'avgWs','minRhm','avgRhm','n99Rn', 'PRRCK_LARG', 'PRRCK_MDDL','LOCTN_ALTT','LOCTN_GRDN',
                                    'CLZN_CD','TPGRP_TPCD','PRDN_FOM_C','SLDPT_TPCD','SLANT_TYP','SCSTX_CD','SLTP_CD','STORUNST','FROR_CD',
                                    'FRTP_CD','KOFTR_GROU','DMCLS_CD','AGCLS_CD','HEIGHT']]
    fillna_test_specific_day2 = test_specific_day2.fillna(0)
    model_pred = xgb2.predict(fillna_test_specific_day2.to_numpy())
    return model_pred

In [ ]:
length_test_input = len(test_input.index)

#day_1 data
for index in range(length_test_input):
    if test_input['umd'][index] == "삼거동":
        input_date = test_input['date'][index]
        input_sgg = test_input['sgg'][index]
        input_umd = "상문동"
    elif test_input['umd'][index] == "어곡동":
        input_date = test_input['date'][index]
        input_sgg = test_input['sgg'][index]
        input_umd = "강서동"
    elif test_input['umd'][index] == "주진동":
        input_date = test_input['date'][index]
        input_sgg = test_input['sgg'][index]
        input_umd = "소주동"
    elif test_input['umd'][index] == "시동":
        input_date = test_input['date'][index]
        input_sgg = test_input['sgg'][index]
        input_umd = "불국동"
    elif test_input['umd'][index] == "덕곡동":
        input_date = test_input['date'][index]
        input_sgg = test_input['sgg'][index]
        input_umd = "지좌동"
    elif test_input['umd'][index] == "조와동":
        input_date = test_input['date'][index]
        input_sgg = test_input['sgg'][index]
        input_umd = "상망동"
    else:
        input_date = test_input['date'][index]
        input_sgg = test_input['sgg'][index]
        input_umd = test_input['umd'][index]
    test_input['1day_yn'][index] = day1_data_test(input_sgg, input_umd, input_date)
        

#day_2 data
for index in range(length_test_input):
    if test_input['umd'][index] == "삼거동":
        input_date = test_input['date'][index]
        input_sgg = test_input['sgg'][index]
        input_umd = "상문동"
    elif test_input['umd'][index] == "어곡동":
        input_date = test_input['date'][index]
        input_sgg = test_input['sgg'][index]
        input_umd = "강서동"
    elif test_input['umd'][index] == "주진동":
        input_date = test_input['date'][index]
        input_sgg = test_input['sgg'][index]
        input_umd = "소주동"
    elif test_input['umd'][index] == "시동":
        input_date = test_input['date'][index]
        input_sgg = test_input['sgg'][index]
        input_umd = "불국동"
    elif test_input['umd'][index] == "덕곡동":
        input_date = test_input['date'][index]
        input_sgg = test_input['sgg'][index]
        input_umd = "지좌동"
    elif test_input['umd'][index] == "조와동":
        input_date = test_input['date'][index]
        input_sgg = test_input['sgg'][index]
        input_umd = "상망동"
    else:
        input_date = test_input['date'][index]
        input_sgg = test_input['sgg'][index]
        input_umd = test_input['umd'][index]
    test_input['2day_yn'][index] = day2_data_test(input_sgg, input_umd, input_date)


test_input.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,date,sd,sgg,umd,1day,1day_yn,2day,2day_yn
0,2020-06-10,경상남도,사천시,곤명면,2020-06-11,0.0,2020-06-12,0.0
1,2020-06-11,경상남도,사천시,곤명면,2020-06-12,0.0,2020-06-13,0.0
2,2020-06-12,경상남도,사천시,곤명면,2020-06-13,0.0,2020-06-14,0.0
3,2020-06-10,경상남도,사천시,곤양면,2020-06-11,0.0,2020-06-12,0.0
4,2020-06-11,경상남도,사천시,곤양면,2020-06-12,0.0,2020-06-13,0.0


In [ ]:
test_input["1day_yn"].value_counts()

0.0    3397
1.0      13
Name: 1day_yn, dtype: int64

In [ ]:
test_input["2day_yn"].value_counts()

0.0    3410
Name: 2day_yn, dtype: int64

In [ ]:
test_input.to_csv("xgb_1v30_model.csv", index=False, encoding='cp949')